<h1 align=center><font size = 8>Segmenting and Clustering Neighborhoods in Toronto, Canada</font></h1>

First of all, we have to import libraries and the dataset

## Step 1: Import libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 92kB 18.7MB/s eta 0:00:01
Libraries imported.


## Step 2: Download dataset

In this case, the dataset that we have is given by Wikipedia, which is a table of neighborhoods and their post code. This table is given in HTML, so we have to convert it to a Panda Dataframe.

To do that, I create a string with the HTML code.

In [65]:
from bs4 import BeautifulSoup

req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[0]

df_can = pd.read_html(str(table))

neighborhood=pd.DataFrame(df_can[0])

In [66]:
df_can = df_can[0].sort_values(by = ["Postal code"])
df_can.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
9,M1B,Scarborough,Malvern / Rouge
18,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
27,M1E,Scarborough,Guildwood / Morningside / West Hill
36,M1G,Scarborough,Woburn


In [49]:
df_can.dropna(inplace = True)
df_can.drop(df_can.loc[df_can['Borough']=='Not assigned'].index, inplace=True)
df_can.head()

,Postal code,Borough,Neighborhood
9,M1B,Scarborough,Malvern / Rouge
18,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
27,M1E,Scarborough,Guildwood / Morningside / West Hill
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae


In [50]:
df_can.sort_values(by=["Postal code"],axis=0)
df_can = df_can.reset_index(drop=True)

Another criteria is to group the neighborhoods by post code, which is already done in the wikipedia table given. The problem is that is separated in each row with a slash "/", so there is need to replace it with a comma " ,". Then, a reset index is done.

In [51]:
df_can['Neighborhood'] = df_can['Neighborhood'].str.replace(" /",",")
df_can.reset_index(drop=True, inplace = True)
df_can.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


And there we have it. The dataframe is sorted by Postal code and every neighborhood is in the row of each postal code. Let's see the shape of the dataframe:

In [47]:
df_can.shape

(180, 3)

In [53]:
!pip install geocoder==1.5.0
import geocoder # import geocoder



     |████████████████████████████████| 51kB 15.7MB/s eta 0:00:01


NameError: name 'postal_code' is not defined

In [64]:
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('Toronto, CA')
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

IndexError: list index out of range